# Pipeline

In [1]:
%load_ext autoreload
%autoreload 2

In [11]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import setting
from sklearn import set_config

from src import viz
from src.common import util

In [13]:
sns.set_theme(style="whitegrid")

In [4]:
df_train, df_test = util.load_data()

In [16]:
from sklearn.ensemble import RandomTreesEmbedding, HistGradientBoostingRegressor
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

